In [1]:
import os
import sys
import json
from tqdm import tqdm_notebook as tqdm

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
w_dir

'/work/nlu_IR'

In [2]:
sys.path.append(w_dir+'/nlu_IR')

In [3]:
from nlu_IR import config
from nlu_IR import std

In [5]:
data = std.json_load(config.COS_DEV)

In [6]:
data[0]['QUESTIONS'][0]['QTEXT_CN']

'特穆拉·莫里森演什么电影？'

In [7]:
from nlu_IR.lucene_search.cosQA_search import CosQASearcher
from nlu_IR.evaluation.eval import eval_sp

In [8]:
mySearcher = CosQASearcher('zh')

In [9]:
mySearcher.search('特穆拉·莫里森演什么电影？', 5)

[('Q2155524',
  'River Queen',
  '河皇后是2005年由文森特·沃德执导，萨曼莎·莫顿、基弗·萨瑟兰、克里夫·柯蒂斯、特穆拉·莫里森和斯蒂芬·雷亚主演的新西兰-英国战争戏剧电影。这部电影的评价参差不齐，但在新西兰的票房表现良好。',
  7.7007856369018555),
 ('Q1184001',
  'What a Girl Wants',
  '女孩想要什么可以指：“女孩想要什么”（克里斯蒂娜·阿奎莱拉之歌），1999女孩想要什么（电影），由阿曼达·拜恩斯主演的2003年电影“女孩想要什么”（B2K之歌），2003年“女孩想要什么”，一首4分钟前为穆齐克创作的歌',
  7.653810977935791),
 ('Q529851',
  'Temuera Morrison',
  '特穆拉·德里克·莫里森（1960年12月26日出生）是新西兰演员，他在新西兰肥皂剧《肖特兰街》（Shortland Street）中扮演的Hone Ropata博士首次获得认可。他在1994年的电影《曾经是勇士》和1999年的续集《伤心的人是什么》中饰演杰克·海克，获得了评论界的好评？。他在《星球大战：第二集克隆人的进攻》（2002年）和《星球大战：第二集克隆人的进攻》和《星球大战：第三集西斯的复仇》（2005年）中扮演了詹戈·费特（Jango Fett），从而在国际上享有盛名。他还因在《莫阿纳》（2016）中饰演片名人物之父的崔局长以及在《阿奎曼》（2018）中饰演亚瑟·柯里的父亲托马斯而闻名。',
  7.634599208831787),
 ('Q3536311',
  'Tracker (2011 film)',
  '跟踪器是一部2011年英国-新西兰动作惊悚片，由伊恩·夏普执导，雷·温斯顿和特穆拉·莫里森主演。它设定在1903年的新西兰。',
  7.125396251678467),
 ('Q1759154',
  'The Beautiful Country',
  '美丽的国家是一部以1990年为背景的2004年戏剧电影。这部电影由汉斯·佩特·莫兰执导，达米安·阮、尼克·诺尔特、白玲、周铁金玄、蒂姆·罗斯、安·图、特穆拉·莫里森和约翰·侯赛因主演。剧本由萨比娜·默里（Sabina Murray）根据默里和特伦斯·马利克（Te

In [10]:
import sys

In [23]:
def inspect_cosqa(data):
    num_gold_sp = 0
    valid_data = []
    invalid_data = []
    gold_sp_nums = []
    for d in tqdm(data):
        try:
            assert len(d['QUESTIONS']) == 1, 'There are {} questions!'.format(len(d['QUESTIONS']))
            gold_sp = d['QUESTIONS'][0]['SHINT_']
            num_gold_sp += len(gold_sp)
            gold_sp_nums.append(len(gold_sp))
            valid_data.append(d)
        except Exception as e:
            invalid_data.append(d)
            continue
    print("average_gold_sp:{}".format(num_gold_sp/len(valid_data)))
    print("Valid data:{}".format(len(valid_data)))
    print("Invalid data:{}".format(len(invalid_data)))
    print("max sp_num:{}".format(max(gold_sp_nums)))
    print("min sp_num:{}".format(min(gold_sp_nums)))

In [24]:
inspect_cosqa(data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



average_gold_sp:1.7692932575142162
Valid data:2462
Invalid data:4248
max sp_num:32
min sp_num:0


In [13]:
def search_batch(q_texts, searcher, top_n):
    predictions = []
    
    for q_text in tqdm(q_texts):
        prediction = searcher.search(q_text, top_n)
        predictions.append([p[0] for p in prediction])
    return predictions

In [19]:
def eval_cosqa(data, top_n):
    searcher_cn = CosQASearcher('zh')
    searcher_en = CosQASearcher('en')
    
    cn_q_texts = []
    en_q_texts = []
    golds = []
    for d in tqdm(data):
        try:
            assert len(d['QUESTIONS']) == 1, 'There are {} questions!'.format(len(d['QUESTIONS']))
            cn_q_texts.append(d['QUESTIONS'][0]['QTEXT_CN'])
            en_q_texts.append(d['QUESTIONS'][0]['QTEXT'])
            golds.append(d['QUESTIONS'][0]['SHINT_'])
        except Exception as inst:
#             print(type(inst))    # the exception instance
#             print(inst.args)     # arguments stored in .args
            continue
            
    #search
    cn_predictions = search_batch(cn_q_texts, searcher_cn, top_n)
    en_predictions = search_batch(en_q_texts, searcher_en, top_n)
    
    print("cn: ".format(eval_sp(golds, cn_predictions)))
    print("en: ".format(eval_sp(golds, en_predictions)))

In [26]:
eval_cosqa(data, 50)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.



['Q2155524', 'Q1184001', 'Q529851', 'Q3536311', 'Q1759154', 'Q941654', 'Q4379616', 'Q1199446', 'Q2520460', 'Q5243353', 'Q830773', 'Q4954444', 'Q1402501', 'Q16279537', 'Q943047', 'Q156394', 'Q1508611', 'Q59317', 'Q269326', 'Q661422', 'Q885161', 'Q230084', 'Q7136855', 'Q3566235', 'Q1869916', 'Q832363', 'Q193357', 'Q1056903', 'Q65476839', 'Q1130966', 'Q314485', 'Q1546566', 'Q964771', 'Q1547697', 'Q310637', 'Q1167668', 'Q317026', 'Q234137', 'Q1631702', 'Q4778174', 'Q3986749', 'Q2833067', 'Q1306660', 'Q732397', 'Q7991741', 'Q745041', 'Q643035', 'Q1989718', 'Q454200', 'Q937087']
{'sp_em': 0.0, 'sp_prec': 0.026, 'sp_recall': 0.691, 'sp_f1': 0.048}
cn: 
{'sp_em': 0.0, 'sp_prec': 0.029, 'sp_recall': 0.758, 'sp_f1': 0.054}
en: 
